In [1]:
list('hello')

['h', 'e', 'l', 'l', 'o']

In [4]:
import os
import shutil
import re
import itertools as it
import pandas as pd
os.chdir("/Users/BeckyMarcusMacbook/manual_work")
text_dir = "data/groups/E5/texts"
dest_dir = "data/groups/E5/temp/first_pages"
#dest_dir = "data/groups_cleaned/E5"
def reset():
    rm_ds(text_dir)
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
    shutil.copytree(text_dir,dest_dir)
def rm_ds(dir):
    for file in os.listdir(dir):
        if file==".DS_Store":
            os.remove(os.path.join(dir,file))
## Read folder to df

page_ids = []
texts = []
doc_ids = []
for file in sorted(os.listdir(text_dir)):
    if not ".txt" in file:
        continue
    page_id = file.split(".")[0]
    doc_id = page_id[:-3]
    doc_ids.append(doc_id)
    page_ids.append(page_id)
    with open(os.path.join(text_dir,file),'r') as f:
        text = f.read()
    texts.append(text)


df=pd.DataFrame({"page_id":page_ids,'doc_id':doc_ids,"text":texts})
def get_info(name):
    _,year,num,page = name.split("-")
    return int(year),int(num),int(page)
df['text_path'] = df['page_id'].apply(lambda x: os.path.join(text_dir,x+".txt"))
df['file_name'] = df['page_id']+".txt"
df['pdf_path'] = df['text_path'].apply(lambda x: x.replace("texts","pdfs").replace(".txt",".pdf"))
df[['year','num','page']] = df['page_id'].apply(get_info).to_list()

df

,page_id,doc_id,text,text_path,file_name,pdf_path,year,num,page
0,Economics-1942-0-00,Economics-1942-0,The Conditions of Expansion \nAuthor(s): Sumn...,data/groups/E5/texts/Economics-1942-0-00.txt,Economics-1942-0-00.txt,data/groups/E5/pdfs/Economics-1942-0-00.pdf,1942,0,0
1,Economics-1942-0-01,Economics-1942-0,The American Economic Review \n VOLUME XXXII ...,data/groups/E5/texts/Economics-1942-0-01.txt,Economics-1942-0-01.txt,data/groups/E5/pdfs/Economics-1942-0-01.pdf,1942,0,1
2,Economics-1942-0-02,Economics-1942-0,2 THE AMERICAN ECONOMIC REVIEW [MARCH \n II\n...,data/groups/E5/texts/Economics-1942-0-02.txt,Economics-1942-0-02.txt,data/groups/E5/pdfs/Economics-1942-0-02.pdf,1942,0,2
3,Economics-1942-0-03,Economics-1942-0,1942 ] SLICHTER: THE CONDITIONS OF EXPANSION ...,data/groups/E5/texts/Economics-1942-0-03.txt,Economics-1942-0-03.txt,data/groups/E5/pdfs/Economics-1942-0-03.pdf,1942,0,3
4,Economics-1942-0-04,Economics-1942-0,4 THE AMERICAN ECONOMIC REVIEW [MARCH \n of t...,data/groups/E5/texts/Economics-1942-0-04.txt,Economics-1942-0-04.txt,data/groups/E5/pdfs/Economics-1942-0-04.pdf,1942,0,4
...,...,...,...,...,...,...,...,...,...
648,Economics-1968-0-15,Economics-1968-0,FRIEDMAN: MONETARY POLICY 13 \n self from bei...,data/groups/E5/texts/Economics-1968-0-15.txt,Economics-1968-0-15.txt,data/groups/E5/pdfs/Economics-1968-0-15.pdf,1968,0,15
649,Economics-1968-0-16,Economics-1968-0,"14 THE AMERICAN ECONOMIC REVIEW \n Finally, m...",data/groups/E5/texts/Economics-1968-0-16.txt,Economics-1968-0-16.txt,data/groups/E5/pdfs/Economics-1968-0-16.pdf,1968,0,16
650,Economics-1968-0-17,Economics-1968-0,FRIEDMAN: MONETARY POLICY 15 \n it will be li...,data/groups/E5/texts/Economics-1968-0-17.txt,Economics-1968-0-17.txt,data/groups/E5/pdfs/Economics-1968-0-17.pdf,1968,0,17
651,Economics-1968-0-18,Economics-1968-0,16 THE AMERICAN ECONOMIC REVIEW \n moved in t...,data/groups/E5/texts/Economics-1968-0-18.txt,Economics-1968-0-18.txt,data/groups/E5/pdfs/Economics-1968-0-18.pdf,1968,0,18


In [6]:
def basic_preprocessing(text:str):
    lines = text.split("\n")
    lines_stripped = [l.strip() for l in lines]
    text = "\n".join(lines_stripped)
    
    jstor_split = text.split("This content downloaded from")
    return jstor_split[0]

In [21]:
temp_dir = "data/groups/E5/temp/first_pages"
text_dir = "data/groups/E5/texts"
if os.path.exists(temp_dir):
    shutil.rmtree(temp_dir)
os.makedirs(temp_dir)
title_pages = []
first_page_dir = "data/groups/E5/temp/front_pages"
delete_page_dir = "data/groups/E5/temp/pages_to_delete"
other_page_dir = "data/groups/E5/temp/long_text_pages"
os.makedirs(other_page_dir,exist_ok=True)
os.makedirs(first_page_dir,exist_ok=True)
os.makedirs(delete_page_dir,exist_ok=True)
for file in sorted(os.listdir(text_dir)):
    if file[0]=='.' or file == 'Economics-1960-1-01.txt':
        continue
    disc,year,num,pagetxt = file.split("-")
    page=int(pagetxt[:-4])
    if page<4:
        old_path = os.path.join(text_dir,file)
        new_path = os.path.join(temp_dir,file)
        old_text = open(old_path,'r').read()
        with open(new_path,'w') as f:
            f.write(basic_preprocessing(old_text))
for file in sorted(os.listdir(temp_dir)):
    disc,year,num,pagetxt = file.split("-")
    page=int(pagetxt[:-4])
    if page==0:
        continue
    path = os.path.join(temp_dir,file)
    text = open(path,'r').read()
    author_line =re.search("\n(By .*\n)",text)
    if author_line is not None or file =='Economics-1953-0-01.txt':
        title_pages.append(file)
        new_path=os.path.join(first_page_dir,file)
    elif len(text)<500:
        new_path = os.path.join(delete_page_dir,file)
    else:
        print(f"{file} likely a full page with length {len(text)}")
        new_path = os.path.join(other_page_dir,file)
    shutil.copyfile(path,new_path)


    
        # print(author_line.group(1).strip())
        # print(text.split(author_line.group(1))[1][:100])
        # print()
#1953 first page not found


Economics-1942-0-02.txt likely a full page with length 2963
Economics-1942-0-03.txt likely a full page with length 4365
Economics-1943-0-03.txt likely a full page with length 2766
Economics-1944-0-02.txt likely a full page with length 2820
Economics-1944-0-03.txt likely a full page with length 2962
Economics-1945-0-03.txt likely a full page with length 2738
Economics-1947-0-02.txt likely a full page with length 3010
Economics-1947-0-03.txt likely a full page with length 3013
Economics-1953-0-02.txt likely a full page with length 2943
Economics-1953-0-03.txt likely a full page with length 2883
Economics-1965-0-02.txt likely a full page with length 2860
Economics-1965-0-03.txt likely a full page with length 2986


In [ ]:
built_different = {
    'Economics-1966-0-03.txt':'-R. M. WEAVER',
    'Economics-1953-0-01.txt':'1894-1952',
}

In [32]:
partially_clean = 'data/groups_cleaned/E5'
partially_clean_dir = "data/groups/E5/temp/partially_clean"
if os.path.exists(partially_clean_dir):
    shutil.rmtree(partially_clean_dir)
shutil.copytree(partially_clean,partially_clean_dir,dirs_exist_ok=True)

dest_file = "manual_work/E5_long.txt"
dest_file_edit = "manual_work/E5_long_edited.txt"
if os.path.exists(dest_file):
    os.remove(dest_file)
for file in sorted(os.listdir(partially_clean_dir)):
    path = os.path.join(partially_clean_dir,file)
    text = open(path,'r').read()
    new_text=f"\n\n\n---{file}---\n{text.strip()}"
    with open(dest_file,'a') as f:
        f.write(new_text)
shutil.copyfile(dest_file,dest_file_edit)

'manual_work/E5_long_edited.txt'

### Reading file back

### Break

In [8]:
import re
"""Look from pages 01-03 for all of the texts except 2020 and 2022:
if 01 says "series of photographs":
    mark 02 as picture and 03 as first page
elif 01 doesn't have some indicator that it's a title page (Like the "By...* line"):
    mark 01 as picture and 02 as first page
else: 
    mark 01 as first page
    might also want to check for title page indicator and do something if there's a problem with it
    """
df=pd.DataFrame({"page_id":page_ids,'doc_id':doc_ids,"text":texts})
## Basic info:
def get_info(name):
    _,year,num,page = name.split("-")
    return int(year),int(num),int(page)
df['text_path'] = df['page_id'].apply(lambda x: os.path.join(text_dir,x+".txt"))
df['pdf_path'] = df['text_path'].apply(lambda x: x.replace("texts","pdfs").replace(".txt",".pdf"))
df[['year','num','page']] = df['page_id'].apply(get_info).to_list()
df["length"] = df['text'].apply(len)
df["num_lines"] = df['text'].apply(lambda x: x.count("\n")+1)
df['text'] = df.pop('text')
df['num_pages'] = df.groupby('doc_id')['page'].transform('max')+1
df['is_jstor_cover'] = df['page']==0
## Take care of exception
df.loc[(df['year'].isin([2020,2022]))&(df['page']==0),'is_jstor_cover'] =False
# Getting picture and cover pages
df[["is_photo_intro","is_author_photo","is_first_page"]]=False
for doc in df['doc_id'].unique():
    if '2020' in doc or '2022' in doc:
        continue
    idxes= df.loc[(df['doc_id']==doc)&(df['page']>0)&(df['page']<4)].index
    i1,i2,i3 = idxes
    t1, t2,t3 = df.loc[idxes,"text"].values #type:ignore
    if "series of photographs" in t1:
        df.at[i1,"is_photo_intro"] = True
        df.at[i2,"is_author_photo"] = True
        df.at[i3,"is_first_page"] = True
    elif len(re.findall(r"By[^\*]+\*",t2,re.DOTALL))>0:
        df.at[i1,'is_author_photo'] = True
        df.at[i2,'is_first_page'] = True
    else:
        if len(re.findall(r"By[^\*]+\*",t1,re.DOTALL))==0:
            print(f"Couldn't figure out page for {doc}")
        df.at[i1,"is_first_page"]=True
df

Couldn't figure out page for Economics-1942-0
Couldn't figure out page for Economics-1943-0
Couldn't figure out page for Economics-1947-0
Couldn't figure out page for Economics-1949-0
Couldn't figure out page for Economics-1950-0
Couldn't figure out page for Economics-1952-0
Couldn't figure out page for Economics-1953-0
Couldn't figure out page for Economics-1956-0
Couldn't figure out page for Economics-1957-0
Couldn't figure out page for Economics-1960-0


ValueError: not enough values to unpack (expected 3, got 1)